# Football Scheduler

Here we implement the model descripted at [[1]](#References) and run some test with real teams.

## Imports

In [1]:
import numpy as np
from football_scheduler import FootballScheduler, SymetricScheme

## References:
- [1] G. Durán, E. Mijangos, and M. Frisk, “Scheduling the South American qualifiers to the 2018 FIFA World Cup by integer programming,” European Journal of Operational Research, vol. 262, no. 3, pp. 1035–1048, 2017.
- [2] https://pyscipopt.readthedocs.io/en/latest/index.html